In [105]:

import numpy as np


Three selected fairness metrics selected:

### Calibration
**Measures the difference between predicted probabilities and actual outcomes.**
N: Number of bins, B_i: Each bin, n: Total instances
accuracy(B_i): Accuracy within the bin, confidence(B_i): Average predicted probability within the bin
C = sum(|B_i|/n * |accuracy(B_i) - confidence(B_i)|) for i in 1 to N


### Predictive Parity
**Assesses if positive predictions are made at similar rates across different groups.**
a_1, a_2: Different values of the sensitive attribute
P(Y=1 | A=a_1): Probability of a positive prediction for group a_1
P(Y=1 | A=a_2): Probability of a positive prediction for group a_2
PP = |P(Y=1 | A=a_1) - P(Y=1 | A=a_2)|


### Equalized Odds
**Assesses if true positive rates and false positive rates are equal across different groups.**
TPR: True Positive Rate, FPR: False Positive Rate
a_1, a_2: Different values of the sensitive attribute
Equalized Odds = |TPR_a1 - TPR_a2| + |FPR_a1 - FPR_a2|



In [106]:
def load_confusion_matrices():
    cm_priv = np.load('./confusion_matrix_priv_female.npy')
    tn_priv = cm_priv[0][0]
    fp_priv = cm_priv[0][1]
    fn_priv = cm_priv[1][0]
    tp_priv = cm_priv[1][1]
    cm_unpriv = np.load('./confusion_matrix_unpriv_male.npy')
    tn_unpriv = cm_unpriv[0][0]
    fp_unpriv = cm_unpriv[0][1]
    fn_unpriv = cm_unpriv[1][0]
    tp_unpriv = cm_unpriv[1][1]
    return [[tn_priv, fp_priv, fn_priv, tp_priv], [tn_unpriv, fp_unpriv, fn_unpriv, tp_unpriv]]

load_confusion_matrices()

[[26, 23, 12, 37], [34, 15, 14, 35]]

In [107]:



def calibration(confusion_matrix_priv, confusion_matrix_unpriv):
    """
    Calculate the calibration metric using confusion matrices for privileged and unprivileged groups.

    Args:
    confusion_matrix_priv (list): Confusion matrix for the privileged group [TN, FP, FN, TP].
    confusion_matrix_unpriv (list): Confusion matrix for the unprivileged group [TN, FP, FN, TP].

    Returns:
    list: [privileged group value, unprivileged group value, Absolute difference between unprivileged and privileged group].
    """
    tn_priv, fp_priv, fn_priv, tp_priv = confusion_matrix_priv
    tn_unpriv, fp_unpriv, fn_unpriv, tp_unpriv = confusion_matrix_unpriv

    prob_pos_priv = (tp_priv + fp_priv) / (tp_priv + fp_priv + tn_priv + fn_priv) if (tp_priv + fp_priv + tn_priv + fn_priv) > 0 else 0
    prob_pos_unpriv = (tp_unpriv + fp_unpriv) / (tp_unpriv + fp_unpriv + tn_unpriv + fn_unpriv) if (tp_unpriv + fp_unpriv + tn_unpriv + fn_unpriv) > 0 else 0

    privileged_value = (tp_priv + tn_priv) / (tp_priv + fp_priv + tn_priv + fn_priv) if (tp_priv + fp_priv + tn_priv + fn_priv) > 0 else 0
    unprivileged_value = prob_pos_unpriv

    absolute_difference = np.abs(unprivileged_value - privileged_value)

    return [privileged_value, unprivileged_value, absolute_difference]






In [108]:

def predictive_parity(confusion_matrix_priv, confusion_matrix_unpriv):
    """
    Calculate the Predictive Parity metric using two confusion matrices (for privileged and unprivileged groups).

    Args:
    confusion_matrix_priv (list): List containing [TN, FP, FN, TP] for the privileged group.
    confusion_matrix_unpriv (list): List containing [TN, FP, FN, TP] for the unprivileged group.

    Returns:
    list: [privileged group value, unprivileged group value, Absolute difference between unprivileged and privileged group].
    """
    _, _, fn_priv, tp_priv = confusion_matrix_priv
    _, _, fn_unpriv, tp_unpriv = confusion_matrix_unpriv
    prob_pos_priv = tp_priv / (tp_priv + fn_priv) if (tp_priv + fn_priv) > 0 else 0
    prob_pos_unpriv = tp_unpriv / (tp_unpriv + fn_unpriv) if (tp_unpriv + fn_unpriv) > 0 else 0
    privileged_value = prob_pos_priv if (tp_priv + fn_priv) > 0 else 0
    unprivileged_value = prob_pos_unpriv if (tp_unpriv + fn_unpriv) > 0 else 0
    absolute_difference = np.abs(unprivileged_value - privileged_value)
    return [privileged_value, unprivileged_value, absolute_difference]




In [109]:
def equalized_odds(confusion_matrix_priv, confusion_matrix_unpriv):
    """
    Calculate the Equalized Odds metric using two confusion matrices (for privileged and unprivileged groups).

    Args:
    confusion_matrix_priv (list): List containing [TN, FP, FN, TP] for the privileged group.
    confusion_matrix_unpriv (list): List containing [TN, FP, FN, TP] for the unprivileged group.

    Returns:
    list: [privileged group value, unprivileged group value, Absolute difference between unprivileged and privileged group].
    """
    _, _, fn_priv, tp_priv = confusion_matrix_priv
    _, _, fn_unpriv, tp_unpriv = confusion_matrix_unpriv
    tpr_priv = tp_priv / (tp_priv + fn_priv) if (tp_priv + fn_priv) > 0 else 0
    fpr_priv = 1 - (tp_priv / (tp_priv + fn_priv)) if (tp_priv + fn_priv) > 0 else 0
    tpr_unpriv = tp_unpriv / (tp_unpriv + fn_unpriv) if (tp_unpriv + fn_unpriv) > 0 else 0
    fpr_unpriv = 1 - (tp_unpriv / (tp_unpriv + fn_unpriv)) if (tp_unpriv + fn_unpriv) > 0 else 0
    privileged_value = tpr_priv if (tp_priv + fn_priv) > 0 else 0
    unprivileged_value = tpr_unpriv if (tp_unpriv + fn_unpriv) > 0 else 0
    absolute_difference = np.abs(unprivileged_value - privileged_value)
    return [privileged_value, unprivileged_value, absolute_difference]


In [110]:

# Load confusion matrices
confusion_matrices = load_confusion_matrices()
confusion_matrix_priv, confusion_matrix_unpriv = confusion_matrices



In [111]:
# [privileged group value, unprivileged group value, Absolute difference between unprivileged and privileged group]

In [112]:
methods = {
    "calibration_func":calibration,
    "predictive_parity_func":predictive_parity,
    "equalized_odds_func":equalized_odds
}


In [113]:

scores = {}

priv_matrix,unpriv_matrix = load_confusion_matrices()

 
    

In [114]:
for func_key, func in methods.items():
       scores[func_key]  = func(priv_matrix,unpriv_matrix)
     

scores

{'calibration_func': [0.6428571428571429,
  0.5102040816326531,
  0.13265306122448983],
 'predictive_parity_func': [0.7551020408163265,
  0.7142857142857143,
  0.04081632653061218],
 'equalized_odds_func': [0.7551020408163265,
  0.7142857142857143,
  0.04081632653061218]}

**Calibration:**
Analyzing the calibration metric, I discovered that the privileged group has a calibration value of around 0.64, whereas the unprivileged group has a value of around 0.51. The absolute difference between these numbers is around 0.13. This shows a significant difference in prediction accuracy between the privileged and unprivileged groups. It suggests that the model's projected probability may not correlate equally well with the actual results for both groups.

**Predictive Parity:**
Focusing at the predictive parity measure, I found a privileged group value of around 0.76 and an unprivileged group value of approximately 0.71. The absolute difference is around 0.04. While the difference is slight, it indicates a difference in the frequencies of positive predictions between both sets.

**Equalized Odds:**
For the equalized odds measure, all the privileged and unprivileged groups have values close to 0.76. The absolute difference is again around 0.04. This means that there is a modest disparity in true and false positive rates between the privileged and unprivileged groups.

Overall, these data show that the model may include some bias, particularly in terms of calibration and predictive parity. It is critical to carefully analyze these findings in the light of the specific case and fairness criteria. Further analysis and model tweaks may be required to resolve any discovered inequalities and improve fairness.



